In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from tqdm import tqdm
import glob

In [8]:
conn = mysql.connector.connect(
    host = " database-2.c3iym8yog7ht.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "pethub",
    password = "addinedu5",
    database = "pethub"
)

In [9]:
cursor = conn.cursor()
cursor.execute("SELECT id, region FROM region_pet_regist")
result = cursor.fetchall()

region_list = []

for each in result:
    region_list.append(each[1])
region_list

['종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [44]:
list_dir = glob.glob("../data/*.xls")
df_list = []

for file_name in list_dir:
    df = pd.read_excel(file_name)
    df["지역명"] = [each.split(" ")[1] for each in tqdm(df["소재지"])]
    df = df[["지역명", "구분", "업체명", "소재지"]]
    df.rename(columns={"구분": "category_id", "업체명": "name", "소재지": "address", "지역명": "region_id"}, inplace=True)
    df_list.append(df)

100%|██████████| 86/86 [00:00<00:00, 1387346.71it/s]


In [45]:
merged_data = pd.concat(df_list)
merged_data.head()

,region_id,category_id,name,address
0,종로구,위탁관리,댕구르르,서울특별시 종로구 평창동 483번지 1호
1,종로구,위탁관리,펫힐링센터,서울특별시 종로구 부암동 279번지 16호
2,종로구,위탁관리,강아지대학로,서울특별시 종로구 동숭동 1번지 139호 한양빌딩
3,종로구,위탁관리,경희궁바른동물병원,서울특별시 종로구 홍파동 42번지 13호 경희궁자이 2단지
4,종로구,위탁관리,북악동물병원,서울특별시 종로구 홍지동 127번지 7호


In [46]:
merged_data["region_id"] = [region_list.index(each)+1 for each in tqdm(merged_data["region_id"])]
merged_data.head()

100%|██████████| 2452/2452 [00:00<00:00, 1334773.97it/s]


,region_id,category_id,name,address
0,1,위탁관리,댕구르르,서울특별시 종로구 평창동 483번지 1호
1,1,위탁관리,펫힐링센터,서울특별시 종로구 부암동 279번지 16호
2,1,위탁관리,강아지대학로,서울특별시 종로구 동숭동 1번지 139호 한양빌딩
3,1,위탁관리,경희궁바른동물병원,서울특별시 종로구 홍파동 42번지 13호 경희궁자이 2단지
4,1,위탁관리,북악동물병원,서울특별시 종로구 홍지동 127번지 7호


In [47]:
category_mapping = {
    "위탁관리 ": 1,
    "미용 (일반)": 2,
    "미용 (자동차이용)": 2,
    "전시 ": 3
}
merged_data["category_id"] = merged_data["category_id"].replace(category_mapping)
merged_data.head()

/var/folders/1g/mh4r7xrj5gg2lcbjp9rv33fr0000gn/T/ipykernel_12285/3403900846.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data["category_id"] = merged_data["category_id"].replace(category_mapping)


,region_id,category_id,name,address
0,1,1,댕구르르,서울특별시 종로구 평창동 483번지 1호
1,1,1,펫힐링센터,서울특별시 종로구 부암동 279번지 16호
2,1,1,강아지대학로,서울특별시 종로구 동숭동 1번지 139호 한양빌딩
3,1,1,경희궁바른동물병원,서울특별시 종로구 홍파동 42번지 13호 경희궁자이 2단지
4,1,1,북악동물병원,서울특별시 종로구 홍지동 127번지 7호


In [48]:
sql = "INSERT INTO pet_infra (region_id, category_id, name, address) VALUES (%s, %s, %s, %s)"
for i, row in merged_data.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    conn.commit()

(1, 1, '댕구르르', '서울특별시 종로구 평창동 483번지 1호')
(1, 1, '펫힐링센터', '서울특별시 종로구 부암동 279번지 16호')
(1, 1, '강아지대학로', '서울특별시 종로구 동숭동 1번지 139호 한양빌딩')
(1, 1, '경희궁바른동물병원', '서울특별시 종로구 홍파동 42번지 13호 경희궁자이 2단지')
(1, 1, '북악동물병원', '서울특별시 종로구 홍지동 127번지 7호')
(1, 1, '서촌도그살롱', '서울특별시 종로구 필운동 290번지')
(2, 1, '강놀애견유치원', '서울특별시 중구 황학동 1780번지 의성빌딩')
(2, 1, '디어코코 별관', '서울특별시 중구 신당동 376번지 대동빌딩')
(2, 1, '힐스타운 동물병원', '서울특별시 중구 신당동 844번지 남산타운')
(2, 1, '디어코코', '서울특별시 중구 신당동 338번지 3호 디오빌딩')
(2, 1, '동물보육원', '서울특별시 중구 신당동 372번지 434호')
(2, 1, '강아지 마법사 골드맘', '서울특별시 중구 신당동 282번지 33호')
(2, 1, '애뚜애뚜', '서울특별시 중구 신당동 47번지 11호')
(2, 1, '미래지동물의료센터', '서울특별시 중구 신당동 368번지 19호')
(2, 1, '남산동물병원', '서울특별시 중구 신당동 844번지 남산타운')
(2, 1, 'COZY', '서울특별시 중구 신당동 294번지 67호')
(3, 1, '너즐리', '서울특별시 용산구 한남동 657번지 103호')
(3, 1, '멍키즈', '서울특별시 용산구 한남동 113번지 2호')
(3, 1, '멍멍샵 플러스', '서울특별시 용산구 이태원동 92번지 9호')
(3, 1, '안녕네모', '서울특별시 용산구 이촌동 300번지 23호 한강맨션')
(3, 1, '호텔 독스', '서울특별시 용산구 문배동 3번지 3호 이안용산프리미어')
(3, 1, '플러피캣(fluffy cat)', '서울특별시 용산구 이태원동 171번지 29호')
(3, 1, 